# # Automated Essay Scoring

The project goal is to create a model that is able to do Automated Essay Scoring


This data set is from Kaggle sponsored by the The William and Flora Hewlett Foundation (Hewlett) 
link to the dataset: https://www.kaggle.com/c/asap-aes



Authors: Legayada, Fritz ; Gorope, Patrick ; Dauberman Jarod

## Import Packages and Libraries needed for the Project

In [1]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error, r2_score,cohen_kappa_score,classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import dataset
df = pd.read_csv("training_set_rel3.tsv", sep='\t', encoding='ISO-8859-1');

In [3]:
# check the head of the dataset
df.head()

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12976 entries, 0 to 12975
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   essay_id        12976 non-null  int64  
 1   essay_set       12976 non-null  int64  
 2   essay           12976 non-null  object 
 3   rater1_domain1  12976 non-null  int64  
 4   rater2_domain1  12976 non-null  int64  
 5   rater3_domain1  128 non-null    float64
 6   domain1_score   12976 non-null  int64  
 7   rater1_domain2  1800 non-null   float64
 8   rater2_domain2  1800 non-null   float64
 9   domain2_score   1800 non-null   float64
 10  rater1_trait1   2292 non-null   float64
 11  rater1_trait2   2292 non-null   float64
 12  rater1_trait3   2292 non-null   float64
 13  rater1_trait4   2292 non-null   float64
 14  rater1_trait5   723 non-null    float64
 15  rater1_trait6   723 non-null    float64
 16  rater2_trait1   2292 non-null   float64
 17  rater2_trait2   2292 non-null  

In [5]:
df.describe()

,essay_id,essay_set,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,rater1_trait1,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
count,12976.000000,12976.000000,12976.000000,12976.000000,128.000000,12976.000000,1800.000000,1800.000000,1800.000000,2292.000000,...,2292.000000,2292.000000,723.000000,723.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000
mean,10295.395808,4.179485,4.127158,4.137408,37.828125,6.800247,3.333889,3.330556,3.333889,2.444154,...,2.635689,2.710297,3.777317,3.589212,3.945312,3.890625,4.078125,3.992188,3.843750,3.617188
std,6309.074105,2.136913,4.212544,4.264330,5.240829,8.970705,0.729103,0.726807,0.729103,1.211730,...,1.142566,1.045795,0.689401,0.693256,0.643668,0.630390,0.622535,0.509687,0.538845,0.603417
min,1.000000,1.000000,0.000000,0.000000,20.000000,0.000000,1.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,1.000000,1.000000,2.000000,2.000000,2.000000,3.000000,2.000000,2.000000
25%,4438.750000,2.000000,2.000000,2.000000,36.000000,2.000000,3.000000,3.000000,3.000000,2.000000,...,2.000000,2.000000,3.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000
50%,10044.500000,4.000000,3.000000,3.000000,40.000000,3.000000,3.000000,3.000000,3.000000,2.000000,...,2.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
75%,15681.250000,6.000000,4.000000,4.000000,40.000000,8.000000,4.000000,4.000000,4.000000,3.000000,...,4.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
max,21633.000000,8.000000,30.000000,30.000000,50.000000,60.000000,4.000000,4.000000,4.000000,6.000000,...,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,5.000000,5.000000


## Preprocessing of Data Phase

In [6]:
# remove other columns
df = df.drop(df.iloc[:,7:],axis = 1) 
df = df.drop(['rater1_domain1', 'rater2_domain1'], axis = 1)
df = df.drop(['rater3_domain1'], axis = 1)

In [7]:
# check the head of the dataset
df.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


In [8]:
# replace tags with words
"""
df['essay'] = df['essay'].str.replace("@CAPS", "caps")
df['essay'] = df['essay'].str.replace("@LOCATION", "location")
df['essay'] = df['essay'].str.replace("@ORGANIZATION", "organization")
df['essay'] = df['essay'].str.replace("@PERSON", "person")
df['essay'] = df['essay'].str.replace("@DATE", "date")
df['essay'] = df['essay'].str.replace("@TIME", "time")
df['essay'] = df['essay'].str.replace("@MONEY", "money")
"""


tlist = ["@CAPS","@LOCATION","@ORGANIZATION","@PERSON","@DATE","@TIME","@MONEY"]
wlist = ["caps","location","organization","person","date","time","money"]
ctr = 0
for i in tlist:
    df['essay'] = df['essay'].str.replace(i,wlist[ctr])
    # print(i,wlist[ctr]) checking
    ctr+=1
#print(ctr)

In [9]:
print(df["essay"][0])

Dear local newspaper, I think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people, helps us learn about the globe(astronomy) and keeps us out of troble! Thing about! Dont you think so? How would you feel if your teenager is always on the phone with friends! Do you ever time to chat with your friends or buisness partner about things. Well now - there's a new way to chat the computer, theirs plenty of sites on the internet to do so: organization1, organization2, caps1, facebook, myspace ect. Just think now while your setting up meeting with your boss on the computer, your teenager is having fun on the phone not rushing to get off cause you want to use it. How did you learn about other countrys/states outside of yours? Well I have by computer/internet, it's a new way to learn about what going on in our time! You might think your child spends a lot of time on the computer, but ask them so question about the economy, s

## Features Engineering Phase

In [11]:
# Number of words in the essay
def NumWords(essay):
    
    cleanEssay = re.sub(r'\W', ' ', essay)
    words = nltk.word_tokenize(cleanEssay)
    return len(words) # return the length of words


# Number of characters in the essay
def NumChar(essay):
    cleanEssay = re.sub(r'\s', '', str(essay).lower())
    return len(cleanEssay) #


# Number of sentences in the essay
def NumSentences(essay):
    
    sentences = nltk.sent_tokenize(essay)
    return len(sentences)


# Extract and Apply Features to essay
def ExtractFeatures(data):
    
    features = data.copy()
    features['NumWords'] = features['essay'].apply(NumWords)
    features['NumChar'] = features['essay'].apply(NumChar)
    features['NumSentences'] = features['essay'].apply(NumSentences)
    return features

ProjectFeatures = ExtractFeatures(df[df['essay_set'] == 1])

print(ProjectFeatures)

      essay_id  essay_set                                              essay  \
0            1          1  Dear local newspaper, I think effects computer...   
1            2          1  Dear caps1 caps2, I believe that using compute...   
2            3          1  Dear, caps1 caps2 caps3 More and more people u...   
3            4          1  Dear Local Newspaper, caps1 I have found that ...   
4            5          1  Dear location1, I know having computers has a ...   
...        ...        ...                                                ...   
1778      1783          1  Dear caps1, caps2 several reasons on way I tha...   
1779      1784          1  Do a adults and kids spend to much time on the...   
1780      1785          1  My opinion is that people should have computer...   
1781      1786          1  Dear readers, I think that its good and bad to...   
1782      1787          1  Dear - Local Newspaper I agree thats computers...   

      domain1_score  NumWords  NumChar 

In [12]:
# remove stop words in essay
#stop_words = stopwords.words('english')
#df['Stop_Words'] = df['essay'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [13]:
# tokenize words in for stop_Words
# df['Tokenize_Words'] = df.apply(lambda row: nltk.word_tokenize(row["Stop_Words"]), axis=1)

In [14]:
df.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear caps1 caps2, I believe that using compute...",9
2,3,1,"Dear, caps1 caps2 caps3 More and more people u...",7
3,4,1,"Dear Local Newspaper, caps1 I have found that ...",10
4,5,1,"Dear location1, I know having computers has a ...",8


In [15]:
#df = df.drop(['essay_id','essay', 'rater3_domain1'], axis = 1)

In [16]:
df.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear caps1 caps2, I believe that using compute...",9
2,3,1,"Dear, caps1 caps2 caps3 More and more people u...",7
3,4,1,"Dear Local Newspaper, caps1 I have found that ...",10
4,5,1,"Dear location1, I know having computers has a ...",8


## Training Model Phase

In [17]:
# Vectors Function for getiing Bag of Words (BOW)
def CountVectors(essay): # replaced with Stop_Word
    
    cntvectorizer = CountVectorizer(max_features = 10000, ngram_range=(1, 3), stop_words='english')
    cntvectors = cntvectorizer.fit_transform(essay) # # replaced with Stop_Word
    fn = cntvectorizer.get_feature_names()
    
    return fn, cntvectors


# Split the Stop_Words into train and test sets
fncv, cntvectors = CountVectors(df[df['essay_set'] == 1]['essay'])
xcv = cntvectors.toarray()
ycv = df[df['essay_set'] == 1]['domain1_score'].to_numpy()  # as.matrix to to_numpy

x_train, x_test, y_train, y_test = train_test_split(xcv, ycv, test_size = 0.3, random_state=101)


# Slit the data set into train and test set with 70% for train set and 30% for test set 
# Also Apply features to the dataset
    
x = np.concatenate((ProjectFeatures.iloc[:, 3:].to_numpy(), xcv), axis = 1)   # as.matrix to to_numpy
y = ProjectFeatures['domain1_score'].to_numpy()   # as.matrix to to_numpy

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3,random_state=101)
#X_train, X_test, y_train, y_test = train_test_split(X_cv, y_cv, test_size = 0.3,random_state=101)



## Linear Regression Model

In [18]:
# Linear Regression Mode

linear_regressor = LinearRegression()
linear_regressor.fit(x_train, y_train)
y_pred = linear_regressor.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
varscore =  linear_regressor.score(x_test, y_test)
cks = cohen_kappa_score(np.rint(y_pred), y_test)

# Print the coefficients
print('The Coefficient of Linear Regerssion model: ',linear_regressor.coef_)

# Print The Mean Squared Error
print('The Mean Squared Error of Linear Regeression Model: ', mse)

# Print The Variance Score
# score is between 0 to 1 with 1 being the highest
print('The Variance Score of Linear Regeression Model: ', varscore)

# Print The Cohen’s Kappa Score
# score is between 0 to 1 with 1 being the complete agreement
print('The Cohen Kappa Score of Linear Regeression Model: ', cks)



The Coefficient of Linear Regerssion model:  [ 9.40434848e-01  2.44272100e-04  1.37022650e-04 ... -3.46267528e-03
  1.95419111e-03 -5.61591723e-04]
The Mean Squared Error of Linear Regeression Model:  0.005868462897170392
The Variance Score of Linear Regeression Model:  0.997631809348205
The Cohen Kappa Score of Linear Regeression Model:  1.0


## Logistic Model

In [19]:
# Vectors Function for getiing Bag of Words (BOW)
def CountVectors(essay): # replaced with Stop_Word
    
    cntvectorizer = CountVectorizer(max_features = 10000, ngram_range=(1, 3), stop_words='english')
    cntvectors = cntvectorizer.fit_transform(essay) # # replaced with Stop_Word
    fn = cntvectorizer.get_feature_names()
    
    return fn, cntvectors


# Split the Stop_Words into train and test sets
fncv, cntvectors = CountVectors(df[df['essay_set'] == 1]['essay'])
xcv = cntvectors.toarray()
ycv = df[df['essay_set'] == 1]['domain1_score'].to_numpy()  # as.matrix to to_numpy

x_train, x_test, y_train, y_test = train_test_split(xcv, ycv, test_size = 0.3, random_state=101)


# Slit the data set into train and test set with 70% for train set and 30% for test set 
# Also Apply features to the dataset
    
x = np.concatenate((ProjectFeatures.iloc[:, 3:].to_numpy(), xcv), axis = 1)   # as.matrix to to_numpy
y = ProjectFeatures['domain1_score'].to_numpy()   # as.matrix to to_numpy

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3,random_state=101)
#X_train, X_test, y_train, y_test = train_test_split(X_cv, y_cv, test_size = 0.3,random_state=101)

In [20]:
logistic_model = LogisticRegression()
logistic_model.fit(x_train,y_train)
y_predict = logistic_model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
varscore =  logistic_model.score(x_test, y_test)
cks = cohen_kappa_score(np.rint(y_pred), y_test)

# Print the coefficients
print('The Coefficient of Logistic  model: ',logistic_model.coef_)

# Print The Mean Squared Error
print('The Mean Squared Error of Logisitic  Model: ', mse)

# Print The Variance Score
# score is between 0 to 1 with 1 being the highest
print('The Variance Score of Logistic Model: ', varscore)

# Print The Cohen’s Kappa Score
# score is between 0 to 1 with 1 being the complete agreement
print('The Cohen Kappa Score of Logisitic  Model: ', cks)


The Coefficient of Logistic  model:  [[ 7.67730960e-04 -7.45469220e-04 -1.14826923e-03 ... -4.21532993e-06
  -6.48827567e-08 -5.23388661e-08]
 [ 3.98408961e-06 -3.70034713e-03 -1.45239515e-02 ... -1.71122227e-06
  -3.33185639e-08 -2.71130917e-08]
 [ 7.92921813e-04  3.55565453e-03 -2.93928550e-03 ... -2.01182887e-05
  -4.23961685e-07 -3.22116028e-07]
 ...
 [-1.54167988e-02 -3.76273967e-03  4.22997552e-03 ... -1.17484344e-05
  -1.21136574e-04 -7.58796258e-05]
 [-1.27507202e-02 -4.21036713e-02  1.22570900e-02 ... -9.90148290e-05
   4.59894954e-05  1.83735772e-04]
 [-8.20526478e-03 -8.20814570e-02  2.00660585e-02 ... -2.54412377e-05
   8.41900064e-05 -1.24524303e-05]]
The Mean Squared Error of Logisitic  Model:  0.005868462897170392
The Variance Score of Logistic Model:  0.38878504672897196
The Cohen Kappa Score of Logisitic  Model:  1.0


## Decision Tree Classifier

In [21]:
# Vectors Function for getiing Bag of Words (BOW)
def CountVectors(essay): # replaced with Stop_Word
    
    cntvectorizer = CountVectorizer(max_features = 10000, ngram_range=(1, 3), stop_words='english')
    cntvectors = cntvectorizer.fit_transform(essay) # # replaced with Stop_Word
    fn = cntvectorizer.get_feature_names()
    
    return fn, cntvectors


# Split the Stop_Words into train and test sets
fncv, cntvectors = CountVectors(df[df['essay_set'] == 1]['essay'])
xcv = cntvectors.toarray()
ycv = df[df['essay_set'] == 1]['domain1_score'].to_numpy()  # as.matrix to to_numpy

x_train, x_test, y_train, y_test = train_test_split(xcv, ycv, test_size = 0.3, random_state=101)


# Slit the data set into train and test set with 70% for train set and 30% for test set 
# Also Apply features to the dataset
    
x = np.concatenate((ProjectFeatures.iloc[:, 3:].to_numpy(), xcv), axis = 1)   # as.matrix to to_numpy
y = ProjectFeatures['domain1_score'].to_numpy()   # as.matrix to to_numpy

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3,random_state=101)
#X_train, X_test, y_train, y_test = train_test_split(X_cv, y_cv, test_size = 0.3,random_state=101)

In [22]:
dtree = DecisionTreeClassifier()
dtree.fit(x_train, y_train)
y_predict = dtree.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
varscore =  dtree.score(x_test, y_test)
cks = cohen_kappa_score(np.rint(y_pred), y_test)

# Print the coefficients
#print('The Coefficient of Decision Tree Classifier: ', dtree.coef_)

# Print The Mean Squared Error
print('The Mean Squared Error of Decision Tree Classifier: ', mse)

# Print The Variance Score
# score is between 0 to 1 with 1 being the highest
print('The Variance Score of Decision Tree Classifier: ', varscore)

# Print The Cohen’s Kappa Score
# score is between 0 to 1 with 1 being the complete agreement
print('The Cohen Kappa Score of Decision Tree Classifier: ', cks)

The Mean Squared Error of Decision Tree Classifier:  0.005868462897170392
The Variance Score of Decision Tree Classifier:  1.0
The Cohen Kappa Score of Decision Tree Classifier:  1.0
